In [1]:
import json

from collections import OrderedDict
from pprint import pprint

import httplib2
import numpy as np
import scipy.optimize

from apiclient.discovery import build
from oauth2client import tools
from oauth2client.client import OAuth2WebServerFlow
from oauth2client.file import Storage

In [2]:
route = {}
route['Morning'] = OrderedDict(
    {'Main St': [25, 40, 30],
     '40 mph Rt. 29': [30, 50, 40],
     '40 mph after circle': [30, 50, 40],
     '45 mph Rt. 29': [40, 50, 45],
     '50 mph Rt. 29': [45, 60, 50],
     'Ferry St': [25, 40, 35],
     'Broadway': [25, 40, 30],
     'Up to Saratoga Apple': [25, 40, 35],
     '45 mph Saratoga Apple': [40, 55, 45],
     '55 mph corn maze': [45, 60, 50],
     'Grangerville': [30, 50, 40],
     'Rt. 29': [45, 60, 50],
     '45 mph Saratoga': [40, 55, 45],
     '30 mph Saratoga': [25, 40, 30],
     'Henning Rd': [30, 50, 40],
     'Exit 14': [35, 50, 40],
     'Northway': [60, 70, 60],
     'Exit 11': [25, 40, 35],
     'Round Lake': [25, 40, 35],
     '45 mph Round Lake': [40, 55, 45],
     'Schauber Rd': [40, 55, 45],
     '40 mph Schauber Rd': [40, 50, 40],
     'Ashdown 1': [40, 50, 40],
     'Ashdown 2': [40, 50, 40],
     'Balltown up hill': [40, 50, 45],
     'Balltown in Rexford': [40, 50, 45],
     'Balltown on bridge': [40, 50, 45],
     '45 mph Balltown': [40, 50, 45],
     '40 mph Balltown': [40, 50, 40],
     'River Rd': [35, 50, 40],
    })

In [3]:
route['Afternoon'] = OrderedDict(
    {'River Rd': [35, 50, 40],
     '40 mph Balltown': [40, 50, 40],
     '45 mph Balltown': [40, 50, 45],
     'Balltown on bridge': [40, 50, 45],
     'Balltown in Rexford': [40, 50, 45],
     'Rt. 146 near Balltown': [40, 60, 45],
     '55 mph Rt. 146': [45, 60, 50],
     'Hill by Vent Fitness': [40, 60, 45],
     'Clifton Park': [40, 50, 45],
     'Rt. 146 Halfmoon': [45, 55, 45],
     'Rt. 146 before light': [45, 60, 45],
     'Rt. 146 after light': [45, 60, 45],
     'Rt. 146 curve': [45, 60, 50],
     'Rt. 146 hill': [40, 60, 45],
     '45 mph Rt. 4': [40, 55, 45],
     'Entering Mechanicville': [30, 45, 35],
     'Mechanicville': [25, 40, 30],
     'Rt. 4': [35, 45, 40],
     'Hill in Stillwater': [25, 40, 35],
     'Stillwater': [25, 40, 30],
     'Stillwater Bridge Rd': [25, 40, 35],
     'CR-113': [45, 60, 50],
     'Sarles Ferry': [40, 50, 45],
     'Rt. 40': [45, 60, 50],
     'Hegeman Bridge Rd': [30, 50, 40],
     'Hill St': [25, 40, 30],
    })

In [4]:
dist = {'Morning': 37.6, 'Afternoon': 35.9}

In [5]:
def extract_initial_values(dset, route=route):
    return [bds[2] for bds in route[dset].values()]
x0 = {}
x0['Morning'] = extract_initial_values('Morning')
x0['Afternoon'] = extract_initial_values('Afternoon')

In [6]:
with open('C:\\Users\\watso\\Downloads\\client_id.json') as handle:
    client_json = json.load(handle)
client_id = client_json['installed']['client_id']
client_secret = client_json['installed']['client_secret']

In [7]:
scope = 'https://www.googleapis.com/auth/sheets'
spreadsheet_id = '1CDSz48tIvITNP_0IwJsgojePSeF1BII-KxNQHqQb_JQ'
flow = OAuth2WebServerFlow(client_id, client_secret, scope)

In [8]:
storage = Storage('credentials.dat')
credentials = storage.get()
if credentials is None or credentials.invalid:
    credentials = tools.run_flow(flow, storage)

In [9]:
http = httplib2.Http()
http = credentials.authorize(http)

In [10]:
service = build('sheets', 'v4', http=http)

In [11]:
def store_headers(dset):
    result = service.spreadsheets().values().update(
        spreadsheetId=spreadsheet_id,
        valueInputOption='USER_ENTERED',
        range=f'{dset}!A1',
        body={'values': [['Segment'] + [key for key in route[dset]] + ['MPG', 'Trial 1', 'Trial 2', 'Trial 3']],
              'majorDimension': 'COLUMNS'},
    ).execute()
store_headers('Morning')
store_headers('Afternoon')

In [12]:
def load_dataset(dset, route=route):
    result = service.spreadsheets().values().get(
        spreadsheetId=spreadsheet_id,
        range=f'{dset}!1:{len(route[dset]) + 2}',
        majorDimension='COLUMNS',
    ).execute()
    return {y[0]: y[1:] for y in result['values'][1:] if len(y) == len(route[dset]) + 2}
data = {}
data['Morning'] = load_dataset('Morning')
data['Afternoon'] = load_dataset('Afternoon')

In [13]:
def xl_col_to_name(col_num, col_abs=False):
    col_num += 1  # Change to 1-index.
    col_str = ''
    col_abs = '$' if col_abs else ''
    while col_num:
        remainder = col_num % 26
        if remainder == 0:
            remainder = 26
        col_letter = chr(ord('A') + remainder - 1)
        col_str = col_letter + col_str
        col_num = int((col_num - 1) / 26)
    return col_abs + col_str

In [14]:
ITER = None
NEXT_TRIAL = None
def fun(x, dset):
    global ITER, NEXT_TRIAL
    x = np.array(np.rint(x), dtype=np.int)
    key = f'Trial {ITER}'
    if key in data[dset]:
        y = float(data[dset][key][-1])
        print(f'{dset} {ITER}: MPG for {x} = {y}')
    else:
        NEXT_TRIAL = [ITER] + list(x)
        raise StopIteration
    ITER += 1
    return dist[dset] / y

In [15]:
def opt(dset, route=route):
    global ITER
    ITER = 1
    try:
        return scipy.optimize.minimize(fun, x0[dset], args=(dset,), method='SLSQP',
                                       options={'eps': 5},
                                       bounds=list(x[:2] for x in route[dset].values()))
    except:
        result = service.spreadsheets().values().update(
            spreadsheetId=spreadsheet_id,
            valueInputOption='USER_ENTERED',
            range=f'{dset}!{xl_col_to_name(ITER)}1',
            body={'values': [[f'Trial {NEXT_TRIAL[0]:d}'] + [float(val) for val in NEXT_TRIAL[1:]]],
                  'majorDimension': 'COLUMNS'},
        ).execute()

In [16]:
for dset in ['Morning', 'Afternoon']:
    opt(dset)

Morning 1: MPG for [30 40 40 45 50 35 30 35 45 50 40 50 45 30 40 40 60 35 35 45 45 40 40 40 45
 45 45 45 40 40] = 26.6
Morning 2: MPG for [30 40 40 45 50 35 30 35 45 50 40 50 45 30 40 40 60 35 35 45 45 40 40 40 45
 45 45 45 40 40] = 25.6
Morning 3: MPG for [35 40 40 45 50 35 30 35 45 50 40 50 45 30 40 40 60 35 35 45 45 40 40 40 45
 45 45 45 40 40] = 26.2
Morning 4: MPG for [30 45 40 45 50 35 30 35 45 50 40 50 45 30 40 40 60 35 35 45 45 40 40 40 45
 45 45 45 40 40] = 25.8
Morning 5: MPG for [30 40 45 45 50 35 30 35 45 50 40 50 45 30 40 40 60 35 35 45 45 40 40 40 45
 45 45 45 40 40] = 26.5
Morning 6: MPG for [30 40 40 50 50 35 30 35 45 50 40 50 45 30 40 40 60 35 35 45 45 40 40 40 45
 45 45 45 40 40] = 25.4
Morning 7: MPG for [30 40 40 45 55 35 30 35 45 50 40 50 45 30 40 40 60 35 35 45 45 40 40 40 45
 45 45 45 40 40] = 25.8
Morning 8: MPG for [30 40 40 45 50 40 30 35 45 50 40 50 45 30 40 40 60 35 35 45 45 40 40 40 45
 45 45 45 40 40] = 25.1
Morning 9: MPG for [30 40 40 45 50 35 35 35 45 5